우리는 지금까지 의식하지는 않았지만 당연한 것처럼 딥러닝 실습을 위해 Tensorflow V2 API를 활용해 왔습니다. 하지만 좋은 딥러닝 프레임워크에는 Tensorflow만 있는 것이 아닙니다. 아무래도 Tensorflow의 가장 강력한 경쟁자인 pyTorch를 언급하지 않을 수 없을 것입니다. Tensorflow API와 달리 철저히 파이써닉하고 직관적인 API 설계와 쉬운 사용법으로 pyTorch는 지금도 엄청난 사용자들의 지지를 받고 있습니다. 하지만 Tensorflow도 V2가 되면서 pyTorch가 가진 장점을 대부분 흡수하고, Keras라는 pyTorch와 아주 닮은 API를 Tensorflow의 표준 API로 삼으면서, Google이 가진 분산환경을 다루는 엄청난 기술력과 결합하여 더욱 강력하게 진화한 딥러닝 프레임워크로 진화해 왔습니다.

이번 시간에는 앞으로도 우리가 계속 사용하게 될 Tensorflow(TF) V2 API의 구성상의 개요를 파악하고, 보다 다양하고 깊이있게 Tensorflow를 이해하고 활용할 수 있는 기본기를 갖추고자 합니다. 이를 통해 보다 전문적인 딥러닝 엔지니어로 성장할 수 있는 계기가 되기를 바랍니다.

## TF1의 특징과 문제점
만약 텐서플로우같은 딥러닝 프레임워크가 없었다면, 딥러닝에 몸담은 엔지니어들에의 업무 강도가 얼마나 더해지게 될까요? 아마도 여러분은 딥러닝 프레임워크 없이 간단한 딥러닝 모델을 구현해 본 경험이 있으실 것입니다. (혹시 없다면 꼭 한번은 해보시기를 권합니다.) 정말 단순한 Fully Connected Network 만을 구현하는 데도 꽤나 힘든 과정이었을 것입니다. 그 중 가장 어려운 단계는 아무래도 딥러닝 모델의 그래디언트를 수식으로 구하고 그것을 바탕으로 backward propagation을 구현하는 것일 것입니다. 만약 모델이 훨씬 복잡해진다면 그 복잡한 수식의 gradient를 구하기 위해 엄청나게 복잡한 미분식을 다루어야 할텐데 생각만 해도 아찔한 일일 것입니다.

그러나, 일반적으로 우리는 딥러닝 모델을 구현하기 위해 그 모델의 gradient 수식을 계산하느라 골머리를 싸매지 않습니다. PyTorch 등 다른 딥러닝 프레임워크들도 마찬가지이긴 하지만, Tensorflow는 forward propagation 방향의 모델만 설계하면 그 모델의 gradient를 사전에 미리 구해둘 수 있습니다. 이것을 가능하게 하기 위해 Tensorflow는 초기 V1때부터 독특한 설계 사상을 보유했는데, 그것은 바로 Tensorflow를 거대한 노드-엣지 사이의 **유향 비순환 그래프(Directed Acyclic Graph, DAG)** 로 정의했다는 점입니다. 그렇게 되면 노드와 노드를 연결하는 매 엣지마다 chain-rule을 기반으로 gradient가 역방향으로 전파될 수 있다는 간단하면서도 강력한 아이디어입니다. 이런 방식을 Tensorflow의 **Graph Mode** 라고 합니다.

그러나, 이런 설계가 주는 큰 단점이 하나 있었습니다. Tensorflow V1은 딥러닝 모델을 구성하는 그래프를 그려나가는 부분과, 그 그래프 상에서 연산이 실제 진행되어과는 과정을 엄격하게 분리해 놓았습니다. 거기에 가장 중요한 것이 바로 `session`이라는 개념이었습니다. 그래서 그래프 사이에 벌어지는 모든 연산은 반드시 `session.run()` 안에서 수행되는 것으로 엄격히 제한되었습니다. 물론 이 방식이 주는 대규모 분산환경에서의 확장성과 생산성이라는 장점도 있었지만, Tensorflow V1은 기본적으로 사용하기가 어려웠습니다. 코드도 길고 지저분할 뿐더러, 파이써닉하지 않기 때문에 구현 방식 자체가 난이도가 높았습니다. 무엇보다도, 그래프를 다 만들어놓고 돌려봐야 비로소 모델 구성시의 문제가 드러나는데, 이렇게 문제가 발생했을 때 해결하기가 너무나 어렵고 복잡했기 때문입니다.

## TF2 - Eager mode 수용
그러나 PyTorch는 달랐습니다. PyTorch는 `Eager Mode`라는 것을 제안했는데, 이것은 딥러닝 그래프가 다 그려지지 않아도 얼마든지 부분 실행 및 오류검증이 가능했습니다. 그리고 코드도 간결하고 파이써닉한 설계를 가지고 있었기 때문에 개발자들에게 훨씬 쉽게 다가갈 수 있었습니다. 그래서 마침내 Tensorflow도 Eager Mode를 전격 수용하게 됩니다. 그리고 Keras라는 또하나의 매우 쉽고 간결한 머신러닝 프레임워크를 수용하여 Tensorflow v2의 표준 API 로 삼았습니다. 이런 변화가 Tensorflow의 사용방식을 어떻게 바꾸었는지 간단한 예시코드를 보겠습니다. 예시코드는 텐서플로우 튜토리얼에서 가져왔습니다.
```
# 텐서플로 1.x
outputs = session.run(f(placeholder), feed_dict={placeholder: input})
# 텐서플로 2.0
outputs = f(input)
```
Session.run()에 의존하지 않고, 그래프를 완성하지 않아도 부분적인 실행이 가능하다는 Eager Mode의 장점은 설계, 구현, 디버깅 전과정을 매우 쉽고 직관적이게 만들어 줍니다.

[Tensorflow v2의 다양한 장점](https://www.datasciencecentral.com/profiles/blogs/tensorflow-1-x-vs-2-x-summary-of-changes)
* Ease of use(사용하기 쉬움)
* Eager Execution(즉시 실행)
* Model Building and deploying made easy(설계 구현 용이성)
* The Data pipeline simplified(데이터 파이프라인 단순화)

## Tensorflow2 API
* Sequential
* Functional
* Model Subclassing

### 1. Sequential API
```
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)

model.fit(x, y, epochs=10, batch_size=32)
```
입력부터 출력까지 레이어를 그야말로 sequential하게 차곡차곡 add해서 쌓아나가기만 하면 됨!
[텐서플로우 2.0 사용하기](https://www.tensorflow.org/tutorials/quickstart/beginner)

* 장점 : 초보자가 접근하기에 매우 쉽다
* 단점 : 모델의 입력과 출력이 여러 개인 경우에는 적합하지 않은 모델링 방식. Sequential 모델은 반드시 입력 1가지, 출력 1가지를 전제로 함.

### 2. Functional API
```
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(__원하는 입력값 모양__))
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.fit(x,y, epochs=10, batch_size=32)
```
* `keras.Model`을 사용
* Sequential Model이란 `keras.Model`을 상속받아 확장한 특수사례임!
* Functional API을 활용하면 Sequential Model보다 자유로운 모델링 가능.
* 함수형으로 모델을 구성한다는 것, **입력과 출력을 규정함으로써 모델 전체를 규정한다**
* `Input`이라는 것을 규정합니다. `Input`이 될 수 있는 텐서가 여러개가 될 수 도 있습니다. 그리고 레이어들을 자유롭게 엮어 출력(Output)까지 규정하면 Model이란 바로 inputs와 outputs 만으로 규정됩니다. 정말 Functional하지 않나요?
* [Functional API](https://www.tensorflow.org/guide/keras/functional)

### 3. Subclassing API
```
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()

    def call(self, x):
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)

        return x

model = CustomModel()
model.fit(x,y, epochs=10, batch_size=32)
```
`Subclassing`을 활용하면 제일 자유로운 모델링을 진행할 수 있습니다. 사실 본질적으로는 `Functional`한 접근과 차이가 없습니다. 이것은 `keras.Model`을 상속받은 모델 클래스를 만드는 것이기 때문!
* Sequential Model도 따지고보면 `keras.Model`을 상속받은 모델 클래스의 하나일 뿐.
* keras.Model은 위와 같이 `__init__()`이라는 메소드 안에서 **레이어 구성을 정의.**
* 그리고 `call()`이라는 메소드 안에서 **레이어간 forward propagation을 구현**.
* [Subclassing API](https://www.tensorflow.org/tutorials/quickstart/advanced)

## Automatic differentiation - GradientTape
```
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
```
model.fit()이라는 한줄로 수행 가능한 딥러닝 모델 훈련 과정은 실제로는 어떠했나요?
* Forward Propagation 수행 및 중간 레이어값 저장
* Loss 값 계산
* 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
* 학습 파라미터 업데이트

이런 과정이 TF2 API에는 model.fit()이라는 메소드 안에 모두 추상화되어 감추어져 있습니다.

Tensorflow에서 제공하는 tf.GradientTape는 위와 같이 순전파(forward pass) 로 진행된 모든 연산의 중간 레이어값을 **tape에 기록**하고, 이를 이용해 gradient를 계산한 후 **tape를 폐기**하는 기능을 수행합니다. 
* tf.GradientTape는 이후 그래디언트를 좀더 고급스럽게 활용하는 다양한 기법을 통해 자주 만나게 될 것입니다.
```
# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss
```